In [ ]:
import logging
from indicnlp.tokenize import indic_tokenize
import time
from gensim.models import Word2Vec

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Replace 'kn.txt' with the path to your large Kannada data file
input_file_path = 'kn.txt/kn.txt'
output_file_path = 'tokenized_data.txt'

# Set the batch size
batch_size = 1000

# Open the output file in append mode
with open(output_file_path, 'a', encoding='utf-8') as output_file:
    # Read and tokenize Kannada text in batches
    with open(input_file_path, 'r', encoding='utf-8') as file:
        lines_batch = []

        for i, line in enumerate(file, start=1):
            logging.info(f"Processing line {i}...")

            # Tokenize the line
            tokenized_line = indic_tokenize.trivial_tokenize(line, lang='kn')

            # Add the tokenized line to the batch
            lines_batch.append(' '.join(tokenized_line))

            # Check if the batch size is reached
            if i % batch_size == 0:
                # Write the batch to the output file
                output_file.write('\n'.join(lines_batch) + '\n')
                # Clear the batch for the next set of lines
                lines_batch = []

        # Write any remaining lines in the last batch
        if lines_batch:
            output_file.write('\n'.join(lines_batch) + '\n')

logging.info("Tokenization and writing to file completed.")


In [ ]:
import logging
from gensim.models import Word2Vec
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def read_data_in_batches(filename, batch_size):
    with open(filename, 'r', encoding='utf-8') as file:
        lines = []
        for i, line in enumerate(file):
            lines.append(line.split())
            if i % batch_size == batch_size - 1:
                yield lines
                lines = []
        if lines:
            yield lines

# Create Word2Vec model with batches
word2vec_model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4)
# word2vec_model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4, sg=0)
# by default its sg = 1(skip-gram)

data_filename = 'tokenized_data.txt'
batch_size = 1000  # Adjust the batch size according to your preference

# Build vocabulary
logging.info("Building vocabulary...")
for batch_num, batch in enumerate(read_data_in_batches(data_filename, batch_size), start=1):
    word2vec_model.build_vocab(batch, progress_per=10000)
    logging.info(f"Processed batch {batch_num}, estimated completion time: {datetime.now()}")

# Train the model using batches
logging.info("Training Word2Vec model...")
for epoch in range(3):  # Number of epochs
    for batch_num, batch in enumerate(read_data_in_batches(data_filename, batch_size), start=1):
        word2vec_model.train(batch, total_examples=word2vec_model.corpus_count, epochs=1)
        logging.info(f"Trained on batch {batch_num}, estimated completion time: {datetime.now()}")

# Save the trained Word2Vec model
output_model_path = 'output_word2vec_model.bin'
logging.info("Saving the trained Word2Vec model...")
word2vec_model.save(output_model_path)

logging.info("Word2Vec model training and saving completed.")


In [ ]:
from gensim.models import FastText
from datetime import datetime
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def read_data_in_batches(filename, batch_size):
    with open(filename, 'r', encoding='utf-8') as file:
        lines = []
        for i, line in enumerate(file):
            lines.append(line.split())
            if i % batch_size == batch_size - 1:
                yield lines
                lines = []
        if lines:
            yield lines

# Initialize FastText model with batches
Fasttext_model = FastText(size=100, window=5, min_count=1, workers=4)

data_filename = 'tokenized_data.txt'
batch_size = 1000  # Adjust the batch size according to your preference

# Build vocabulary
logging.info("Building vocabulary...")
for batch_num, batch in enumerate(read_data_in_batches(data_filename, batch_size), start=1):
    Fasttext_model.build_vocab(sentences=batch, progress_per=10000)
    logging.info(f"Processed batch {batch_num}, estimated completion time: {datetime.now()}")

# Train the model using batches
logging.info("Training FastText model...")
for epoch in range(3):  # Number of epochs
    for batch_num, batch in enumerate(read_data_in_batches(data_filename, batch_size), start=1):
        Fasttext_model.train(sentences=batch, total_examples=Fasttext_model.corpus_count, epochs=1)
        logging.info(f"Trained on batch {batch_num}, estimated completion time: {datetime.now()}")

# Save the trained FastText model
output_model_path = 'output_fasttext_model.bin'
logging.info("Saving the trained FastText model...")
Fasttext_model.save(output_model_path)

logging.info("FastText model training and saving completed.")


In [ ]:
# with checkpoints

from transformers import BertModel, BertTokenizer
import torch
import logging
import numpy as np
from datetime import datetime

# Replace 'kn.txt' with the path to your tokenized data file
data_filename = 'kn.txt'
checkpoint_file = 'checkpoint_bert_embeddings.npy'

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Create a list to store BERT embeddings
bert_embeddings = []

def read_data_in_batches(filename, batch_size):
    with open(filename, 'r', encoding='utf-8') as file:
        lines = []
        for i, line in enumerate(file):
            lines.append(line.strip())  # Remove leading/trailing whitespaces
            if i % batch_size == batch_size - 1:
                yield lines
                lines = []
        if lines:
            yield lines

# Function to tokenize and get BERT embeddings for a batch of sentences
def get_bert_embeddings_batch(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    return embeddings

batch_size = 16  # Adjust the batch size according to your preference

# Check if there's a checkpoint file and load the previous embeddings
try:
    previous_embeddings = np.load(checkpoint_file, allow_pickle=True)
    bert_embeddings.extend(previous_embeddings.tolist())
    logging.info(f"Loaded {len(previous_embeddings)} embeddings from the checkpoint.")
except FileNotFoundError:
    logging.info("No checkpoint file found. Starting from the beginning.")

# Iterate through your text data and get BERT embeddings in batches
for batch_num, batch_lines in enumerate(read_data_in_batches(data_filename, batch_size), start=len(bert_embeddings) // batch_size + 1):
    logging.info(f"Processing batch {batch_num}, estimated completion time: {datetime.now()}")
    embeddings_batch = get_bert_embeddings_batch(batch_lines)
    bert_embeddings.append(embeddings_batch)

    # Save checkpoint after every batch
    np.save(checkpoint_file, np.array(bert_embeddings))

# 'bert_embeddings' now contains the BERT embeddings for the sentences in your file

# Optionally, you can save the final embeddings for later use
output_embeddings_bert = 'bert_embeddings.npy'
np.save(output_embeddings_bert, bert_embeddings)

logging.info("BERT embeddings extraction completed.")
